# QA - Basics

In [39]:
import os
from langchain_openai import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [40]:
loader = TextLoader('docs/monologo.txt', encoding="utf-8")
documents = loader.load()

In [14]:
documents

[Document(metadata={'source': 'docs/monologo.txt'}, page_content='Auto o due ruote… è da settimane che mi gira in testa questa domanda. L’auto ha una logica: comoda, versatile, climatizzatore d’estate e aria calda d’inverno.\nPuoi buttarci dentro la spesa, gli amici, perfino i bagagli. Ma ogni volta che mi immagino al volante vedo me stesso bloccato nel traffico,\ncon lo sguardo fisso sul paraurti davanti, le mani che stringono il volante come se stessi aspettando che succeda qualcosa… e non succede mai.\nÈ come vivere metà della vita in una scatola di metallo ferma, parcheggiata o imbottigliata. No, l’auto non è libertà, è routine.\n\nLa moto invece… due ruote sono vento, scia, movimento. È il cuore che accelera quando apri il gas, è la città che si apre come un labirinto da attraversare veloce,\nfluido, senza catene. È libertà che senti sulla pelle, nelle braccia, persino nel rumore cupo del motore che vibra sotto di te. Non c’è paragone: voglio una moto.\n\nPerò… scooter o moto? Lo 

In [41]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 openai_api_key=os.getenv("openai_api_key"))

### Alcuni splitter standard

- **CharacterTextSplitter** → divide il testo in chunk fissi basati su un set di caratteri.
- **RecursiveCharacterTextSplitter** → suddivide ricorsivamente il testo cercando di mantenere i chunk entro una lunghezza massima, privilegiando separatori logici (paragrafi, frasi, parole).
- **NLTKTextSplitter** → utilizza la libreria *NLTK* per individuare frasi da trasformare in chunk.
- **SpacyTextSplitter** → sfrutta la libreria *Spacy* per riconoscere frasi e convertirle in chunk.
- **MarkdownTextSplitter** → analizza la sintassi *Markdown* (intestazioni, liste, blocchi di codice, …) per creare sezioni atomiche.
- **LatexTextSplitter** → interpreta la sintassi *LaTeX* (sezioni, sottosezioni, equazioni) per produrre chunk strutturati.


In [48]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=120, chunk_overlap=40)
splits = text_splitter.split_documents(documents)

splits

[Document(metadata={'source': 'docs/monologo.txt'}, page_content='Auto o due ruote… è da settimane che mi gira in testa questa domanda. L’auto ha una logica: comoda, versatile,'),
 Document(metadata={'source': 'docs/monologo.txt'}, page_content='ha una logica: comoda, versatile, climatizzatore d’estate e aria calda d’inverno.'),
 Document(metadata={'source': 'docs/monologo.txt'}, page_content='Puoi buttarci dentro la spesa, gli amici, perfino i bagagli. Ma ogni volta che mi immagino al volante vedo me stesso'),
 Document(metadata={'source': 'docs/monologo.txt'}, page_content='mi immagino al volante vedo me stesso bloccato nel traffico,'),
 Document(metadata={'source': 'docs/monologo.txt'}, page_content='con lo sguardo fisso sul paraurti davanti, le mani che stringono il volante come se stessi aspettando che succeda'),
 Document(metadata={'source': 'docs/monologo.txt'}, page_content='come se stessi aspettando che succeda qualcosa… e non succede mai.'),
 Document(metadata={'source': 'doc

In [49]:
for i in range(len(splits)):
    print(len(splits[i].page_content))

110
81
116
60
113
66
118
36
116
84
114
84
114
61
115
57
118
57
116
88
117
91
118
64
119
65
115
67
77
107
112
59
82
90
101
113
64
82
115
52
43
97


In [50]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [51]:
retriever = vectorstore.as_retriever()

In [ ]:
# Recupero di un prompt standard dall'Hub di LangChain/LangGraph
# (https://smith.langchain.com/hub)

prompt = hub.pull("rlm/rag-prompt")

human

You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.

Question: {question} 

Context: {context} 

Answer:

In [30]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [52]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    
    {"context": retriever | format_docs,
     "question": RunnablePassthrough()}

    | prompt
    | llm
    | StrOutputParser()
)

In [53]:
rag_chain.invoke("di che colore ha preso la moto?")

'La moto è di colore verde.'

In [54]:
rag_chain.invoke("Ha scelto lo scooter?")

"Non è chiaro se ha scelto lo scooter, ma il testo enfatizza che lo scooter è pratico mentre la moto offre un'esperienza più profonda e coinvolgente. Il suggerimento sembra essere a favore della moto. Pertanto, la scelta finale non è specificata."

In [55]:
rag_chain.invoke("gli piace il traffico?")

"Non sembra che gli piaccia il traffico, anzi, sembra trovarlo pesante e frustrante. Si immagina bloccato nel traffico, il che suggerisce un'esperienza negativa."

In [38]:
result = (retriever | format_docs).invoke("Che moto vuole?")
result

'È la moto che ti parla con sincerità, senza carene a nascondere la sua anima.\n\nfluido, senza catene. È libertà che senti sulla pelle, nelle braccia, persino nel rumore cupo del motore che vibra sotto di te. Non c’è paragone: voglio una moto.\n\nPerò… scooter o moto? Lo scooter è furbo, certo: pratico, ti porta al lavoro senza pensarci troppo, baule sotto la sella, consumi ridotti.\n\n“Eccomi, sono io, guarda!” Deve essere spettacolare, non un verde qualsiasi, ma un verde che ti fa voltare la testa quando passa.\nQuella sarà la mia moto. Verde, naked, 600.'